In [1]:
!pip install faiss-cpu
!pip install faiss-gpu
!pip install transformers peft datasets
!pip install transformers datasets torch
!pip install scikit-learn
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 8.3 MB/s eta 0:00:00


In [3]:
def prepare_finetuning_data(json_file, output_file):
    import json
    with open(json_file, 'r') as file:
        data = json.load(file)

    with open(output_file, 'w') as f:
        for item in data:
            question = item['question']
            answer = item['answer']
            f.write(f"Question: {question}\nAnswer: {answer}\n\n")

# Example usage:
prepare_finetuning_data("train.json", "train.txt")

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
## FOR SINGLE FILE TEST AND TRAIN GENERATION

# from datasets import load_dataset

# # Load the custom dataset
# dataset = load_dataset("text", data_files={"train": "qa_finetune.txt"})

# train_test_split = dataset['train'].train_test_split(test_size=0.2)
# train_dataset = train_test_split['train']
# eval_dataset = train_test_split['test']


# # Tokenize the dataset
# def tokenize_function(examples):
#     tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

#     # Set labels to be the same as input_ids
#     tokenized["labels"] = tokenized["input_ids"].copy()
#     return tokenized

# tokenizer.pad_token = tokenizer.eos_token
# tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

In [6]:
from datasets import load_dataset

# Load the custom datasets from separate files
dataset = load_dataset("text", data_files={"train": "train.txt", "test": "test.txt"})

# Access the train and test datasets directly
train_dataset = dataset['train']
eval_dataset = dataset['test']

# Tokenize the dataset
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    # Set labels to be the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Ensure padding token is set
tokenizer.pad_token = tokenizer.eos_token

# Apply tokenization to the train and eval datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/309 [00:00<?, ? examples/s]

In [7]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./finetuned-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    # gradient_accumulation_steps=4,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)

# Start training (finetuning)
trainer.train()

# Save the finetuned model
trainer.save_model("./finetuned-model")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.045376
2,0.128300,0.044789
3,0.128300,0.044888
4,0.048600,0.044851
5,0.044600,0.045103


In [8]:
finetuned_model = GPT2LMHeadModel.from_pretrained("./finetuned-model")
finetuned_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Generate responses
def generate_finetuned_response(question):
    prompt = f"Question: {question}\nAnswer:"
    inputs = finetuned_tokenizer(prompt, return_tensors="pt")
    outputs = finetuned_model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)
    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage:
response = generate_finetuned_response("Who founded Pittsburgh?")
print(response)
response = generate_finetuned_response("When do Steelers play the Kansas City Chiefs?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your inpu

Question: Who founded Pittsburgh?
Answer: Pittsburgh's first professional sports team was founded in 1876.
Question: When do Steelers play the Kansas City Chiefs?
Answer: Saturday, October 6, 2024


In [16]:
import os
import nltk
nltk.download('punkt')

# Specify the directory containing your text files
context_directory = "./"

# Read all text files from the directory
def load_context_corpus(directory):
    corpus = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read().strip()
                # Split content into sentences or paragraphs for more granular retrieval
                segments = nltk.sent_tokenize(content)
                corpus.extend(segments)  # Add each segment as a separate entry
    return corpus

# Load the context corpus
context_corpus = load_context_corpus(context_directory)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assume `context_corpus` is a list of text documents from your raw text
# context_corpus = [
#     "The Pittsburgh Steelers' colors are black and gold.",
#     "The Pittsburgh Pirates play baseball at PNC Park.",
#     "The next Pirates game is scheduled for October 22nd."
# ]

# Create a TF-IDF vectorizer and fit it to the context corpus
vectorizer = TfidfVectorizer().fit(context_corpus)

# Function to retrieve the most relevant context
def retrieve_context(question, corpus, vectorizer, top_n=1):
    question_vec = vectorizer.transform([question])
    corpus_vec = vectorizer.transform(corpus)
    scores = (corpus_vec * question_vec.T).toarray().flatten()
    best_indices = scores.argsort()[-top_n:][::-1]
    return " ".join([corpus[i] for i in best_indices])

In [ ]:
def truncate_text(text, max_tokens=150):
    tokens = finetuned_tokenizer.tokenize(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return finetuned_tokenizer.convert_tokens_to_string(tokens)

def generate_finetuned_response_with_context(question):
    # Retrieve relevant context
    context = retrieve_context(question, context_corpus, vectorizer)
    print(context)

    truncated_context = truncate_text(context, max_tokens=150)

    # Create the prompt with the retrieved context
    # prompt = f"Context: {truncated_context}\nQuestion: {question}\nAnswer:"
    prompt = f"Here is some background information: {truncated_context}\n\nNow, please answer the following question:\n\n{question}\nAnswer:"

    # Tokenize and generate a response
    inputs = finetuned_tokenizer(prompt, return_tensors="pt")
    outputs = finetuned_model.generate(
        inputs["input_ids"],
        max_length=300,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decode and return the generated response
    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage:
response1 = generate_finetuned_response_with_context("What are Steelers colours?")
print(response1)

# response2 = generate_finetuned_response_with_context("When is the next Pirates game?")
# print(response2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Washington Commanders

Question: Why are Pittsburgh colours Black and Gold?
Here is some background information: Answer: Washington Commanders

Question: Why are Pittsburgh colours Black and Gold?

Now, please answer the following question:

What are Steelers colours?
Answer: Pittsburgh Commandors


In [17]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load a pre-trained embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for your context corpus
context_embeddings = embedder.encode(context_corpus, convert_to_tensor=True)

# Retrieve the most relevant context based on embeddings
def retrieve_context(question, corpus, embeddings, top_n=3):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    # Compute cosine similarities
    similarities = util.pytorch_cos_sim(question_embedding, embeddings)[0]
    # Get the top N most relevant pieces of text
    top_indices = torch.topk(similarities, k=top_n).indices
    return " ".join([corpus[i] for i in top_indices])

# Generate responses with improved retrieval and refined prompt
def generate_finetuned_response_with_context(question):
    # Retrieve relevant context
    context = retrieve_context(question, context_corpus, context_embeddings)

    # Refine the prompt structure
    prompt = (
        f"Background information:\n{context}\n\n"
        f"Based on this information, please answer the following question:\n"
        f"{question}\nAnswer:"
    )

    # Tokenize and generate a response
    inputs = finetuned_tokenizer(prompt, return_tensors="pt")
    outputs = finetuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=500,  # Specify how many new tokens to generate
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decode and return the generated response
    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage:
response1 = generate_finetuned_response_with_context("What are the colours of Pittsburgh Steelers?")
print(response1)

# response2 = generate_finetuned_response_with_context("Why are Pittsburgh colours Black and Gold?")
# print(response2)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Background information:
Answer: Pittsburgh Steelers

Question: Which NHL team represents Pittsburgh? Question: Who founded the Pittsburgh Steelers? Whats the meaning behind the Steelers logo?

Based on this information, please answer the following question:
What are the colours of Pittsburgh Steelers?
Answer: The Pittsburgh Penguins


In [19]:
import json
import string
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer, util
import torch

# Function to preprocess answers (remove punctuation, convert to lowercase)
def preprocess_answer(answer):
    return ''.join([ch for ch in answer if ch not in string.punctuation]).lower().strip()

# Function to compute Exact Match (EM)
def exact_match_score(prediction, ground_truth):
    return int(preprocess_answer(prediction) == preprocess_answer(ground_truth))

# Function to compute F1 score
def f1_score_calculation(prediction, ground_truth):
    # Tokenize the answers
    pred_tokens = preprocess_answer(prediction).split()
    gt_tokens = preprocess_answer(ground_truth).split()

    # Get common tokens between prediction and ground truth
    common_tokens = set(pred_tokens) & set(gt_tokens)

    if len(common_tokens) == 0:
        return 0

    # Calculate precision, recall, and F1 score
    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(gt_tokens)
    if precision + recall == 0:
        return 0
    f1 = 2 * (precision * recall) / (precision + recall)

    return f1

# Load test data
def load_test_data(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        test_data = json.load(file)
    return test_data

# Generate responses using the fine-tuned model
def generate_responses_from_test_data(test_data):
    responses = []
    for entry in test_data:
        question = entry['question']
        response = generate_finetuned_response_with_context(question)
        responses.append({"question": question, "predicted_answer": response, "ground_truth": entry['answer']})
    return responses

# Function to calculate and print average EM and F1 scores
def calculate_evaluation_scores(predictions):
    em_scores = []
    f1_scores = []

    for entry in predictions:
        pred = entry['predicted_answer']
        gt = entry['ground_truth']

        em = exact_match_score(pred, gt)
        f1 = f1_score_calculation(pred, gt)

        em_scores.append(em)
        f1_scores.append(f1)

    average_em = sum(em_scores) / len(em_scores)
    average_f1 = sum(f1_scores) / len(f1_scores)

    print(f"Average Exact Match (EM): {average_em:.2f}")
    print(f"Average F1 Score: {average_f1:.2f}")

# Load your test file (replace 'test_file.json' with your actual test file path)
test_file = "test.json"
test_data = load_test_data(test_file)

# Generate predictions
predictions = generate_responses_from_test_data(test_data)

# Calculate and print evaluation scores
calculate_evaluation_scores(predictions)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Average Exact Match (EM): 0.00
Average F1 Score: 0.06
